In [1]:
## mount drive for access to the files
from google.colab import drive

drive.mount("/content/drive")

## all the drive the files are present in "/content/drive/My Drive"
!ls "/content/drive/My Drive/Beuth Uni/Master Thesis"

import sys
sys.path.append('/content/drive/My Drive/Beuth Uni/Master Thesis/jenga')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Data
'Data Quality in ML Production Systems.pdf'
'Datawig: Missing Value Imputation for Tables.pdf'
 Declaration
 Images
 jenga
 jenga.pdf
 MICE_Multivariate_Imputation_by_Chained_Equations_.pdf


In [2]:
!pip install openml
!pip install pyod

!pip install mxnet autogluon
!pip install mxnet-mkl --pre --upgrade

Requirement already up-to-date: mxnet-mkl in /usr/local/lib/python3.6/dist-packages (1.6.0)


In [3]:
import random
import numpy as np
import pandas as pd

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from jenga.basis import Dataset
from jenga.corruptions.generic import MissingValues, SwappedValues
from jenga.corruptions.numerical import Scaling, GaussianNoise
from jenga.cleaning.ppp import PipelinePerformancePrediction
from jenga.cleaning.outlier_detection import NoOutlierDetection, PyODKNNOutlierDetection, PyODIsolationForestOutlierDetection, AutoGluonOutlierDetection
from jenga.cleaning.imputation import NoImputation, MeanModeImputation, AutoGluonImputation
from jenga.cleaning.clean import Clean

In [4]:
seed = 10

In [5]:
def run_experiment(dataset_name, learner, param_grid, corruptions, fraction, cleaners, num_repetitions, categorical_precision_threshold=0.7, numerical_std_error_threshold=2.0):
    
    ## dataset
    dataset = Dataset(seed, dataset_name)
    
    all_data = dataset.all_data
    attribute_names = dataset.attribute_names
    attribute_types = dataset.attribute_types
    
    ## categorical and numerical features
    categorical_columns = dataset.categorical_columns
    numerical_columns = dataset.numerical_columns
    print(f"Found {len(categorical_columns)} categorical and {len(numerical_columns)} numeric features \n")
    
    ## train and test data
    df_train, lab_train, df_test, lab_test = dataset.get_train_test_data()
    
    
    ## pipeline performance prediction (ppp)
    ppp = PipelinePerformancePrediction(seed, df_train, lab_train, df_test, lab_test, categorical_columns, numerical_columns, learner, param_grid)
    ppp_model = ppp.fit_ppp(df_train)
    
    ## generate corrpted data
    df_corrupted, perturbations, cols_perturbed, summary_col_corrupt = ppp.get_corrupted(df_test, corruptions, fraction, num_repetitions)
    
    ## cleaning
    clean = Clean(df_train, df_corrupted, categorical_columns, numerical_columns, categorical_precision_threshold, numerical_std_error_threshold, ppp, ppp_model, cleaners)
    df_cleaned, corrupted_score_ppp, best_cleaning_score, cleaner_scores_ppp, summary_cleaners = clean(df_train, df_corrupted)
    
    ## results
    result = {
        'ppp_score_model': ppp.predict_score_ppp(ppp_model, df_test),
        'ppp_score_corrupted': corrupted_score_ppp,
        'ppp_score_cleaned': best_cleaning_score,
        'ppp_scores_cleaners': cleaner_scores_ppp
    }
#     print('\n'.join([f'{key}:{val}' for key, val in result.items()]))
    
    ## summary
    summary = {
        'dataset': dataset_name,
        'model': learner,
        'corruptions': summary_col_corrupt,
        'cleaners': summary_cleaners,
        'result': result
    }
#     print('\n\n\n\n'.join([f'{key}:{val}' for key, val in summary.items()]))
    
    return summary #summary_col_corrupt, result

In [6]:
datasets = [
    'parkinsons',
    'heart-statlog',
    'credit-g'
]

In [7]:
## model parameters
## models is a dict where key = leaner & value = param_grid
models = {SGDClassifier(loss='log'): {'learner__max_iter': [500, 1000, 5000], 
                                         'learner__penalty': ['l2', 'l1', 'elasticnet'], 
                                         'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
                                        }#, 
          # RandomForestClassifier():{'learner__n_estimators': [100, 200, 500], 
          #                           'learner__max_depth': [5, 10, 15]
          #                          }
         }

## make dict of multiple leraners and corresponding param_grids

In [8]:
corruptions = [[MissingValues]] #, [Scaling], [GaussianNoise]]

In [9]:
fractions = [0.15]#, 0.25, 0.5, 0.75, 0.9]

In [10]:
cleaners = [
    (NoOutlierDetection, MeanModeImputation),
    (PyODKNNOutlierDetection, MeanModeImputation),
    # (PyODKNNOutlierDetection, AutoGluonImputation),
    (PyODIsolationForestOutlierDetection, MeanModeImputation),
    # (PyODIsolationForestOutlierDetection, AutoGluonImputation),
    (AutoGluonOutlierDetection, AutoGluonImputation)
]

In [11]:
%%time
for _ in range(1):
  print("\n\n..................................ITERATION..................................\n")
  ind_results = []
  
  for dataset in datasets:
    for learner, param_grid in models.items():
      for corruption in corruptions:
        for fraction in fractions:
          ind_results.append(run_experiment(dataset, learner, param_grid, corruption, fraction, cleaners, 100))



..................................ITERATION..................................



Data pickle file already exists and is up to date.


Dataset: parkinsons
Found 0 categorical and 22 numeric features 

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 174 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    3.8s finished



Generating corrupted training data on 39 rows... 

	perturbation: MissingValues: {'column': 'V2', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MNAR'}
	perturbation: MissingValues: {'column': 'V16', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MAR'}
	perturbation: MissingValues: {'column': 'V1', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MNAR'}
	perturbation: MissingValues: {'column': 'V15', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MCAR'}
	perturbation: MissingValues: {'column': 'V9', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MAR'}
	perturbation: MissingValues: {'column': 'V6', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MNAR'}
	perturbation: MissingValues: {'column': 'V17', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MCAR'}
	perturbation: MissingValues: {'column': 'V11', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MAR'}
	perturbation: MissingValues: {'column': 'V8', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MCAR'}
	p

...

Categorical precision threshold: 0.7
Numerical Std Error threshold: 2.0
Predictors: {'checking_status': <autogluon.task.tabular_prediction.predictor.TabularPredictor object at 0x7f6e1e3f6e80>, 'credit_history': <autogluon.task.tabular_prediction.predictor.TabularPredictor object at 0x7f6e27d1c240>, 'purpose': <autogluon.task.tabular_prediction.predictor.TabularPredictor object at 0x7f6e27aed400>, 'savings_status': <autogluon.task.tabular_prediction.predictor.TabularPredictor object at 0x7f6e292f8390>, 'employment': <autogluon.task.tabular_prediction.predictor.TabularPredictor object at 0x7f6e28030588>, 'personal_status': <autogluon.task.tabular_prediction.predictor.TabularPredictor object at 0x7f6e27db1860>, 'other_parties': <autogluon.task.tabular_prediction.predictor.TabularPredictor object at 0x7f6e29f84940>, 'property_magnitude': <autogluon.task.tabular_prediction.predictor.TabularPredictor object at 0x7f6e2bb095f8>, 'other_payment_plans': <autogluon.task.tabular_prediction.predict

Evaluation: accuracy on test data: 0.445
/usr/local/lib/python3.6/dist-packages/autogluon/utils/tabular/ml/learner/abstract_learner.py:360: UserWarning: There are 2 (out of 198) evaluation datapoints for which the label is missing. AutoGluon removed these points from the evaluation, which thus may not be entirely representative. You should carefully study why there are missing labels in your evaluation data.
  warnings.warn(f"There are {n_missing} (out of {len(y_true)}) evaluation datapoints for which the label is missing. "
Evaluation: root_mean_squared_error on test data: 7.699521681198704
Evaluation: accuracy on test data: 0.69
Evaluation: accuracy on test data: 0.215
/usr/local/lib/python3.6/dist-packages/autogluon/utils/tabular/ml/learner/abstract_learner.py:360: UserWarning: There are 3 (out of 197) evaluation datapoints for which the label is missing. AutoGluon removed these points from the evaluation, which thus may not be entirely representative. You should carefully study why

Cleaner: {'outlier_detection': AutoGluonOutlierDetection, 'imputation': AutoGluonImputation}: {'roc_auc_acore': 0.809724170172978, 'classification_report': {'bad': {'precision': 0.6666666666666666, 'recall': 0.5161290322580645, 'f1-score': 0.5818181818181819, 'support': 62}, 'good': {'precision': 0.8026315789473685, 'recall': 0.8840579710144928, 'f1-score': 0.8413793103448277, 'support': 138}, 'accuracy': 0.77, 'macro avg': {'precision': 0.7346491228070176, 'recall': 0.7000935016362786, 'f1-score': 0.7115987460815048, 'support': 200}, 'weighted avg': {'precision': 0.7604824561403509, 'recall': 0.77, 'f1-score': 0.7609153605015675, 'support': 200}}}
Column credit_history: Num NaNs: Before: 0, Now: 31
Column purpose: Num NaNs: Before: 0, Now: 18
Column personal_status: Num NaNs: Before: 0, Now: 2
Column other_parties: Num NaNs: Before: 0, Now: 15
Column property_magnitude: Num NaNs: Before: 0, Now: 1
Column other_payment_plans: Num NaNs: Before: 0, Now: 38
Column housing: Num NaNs: Befor

Evaluation: accuracy on test data: 0.445
/usr/local/lib/python3.6/dist-packages/autogluon/utils/tabular/ml/learner/abstract_learner.py:360: UserWarning: There are 2 (out of 198) evaluation datapoints for which the label is missing. AutoGluon removed these points from the evaluation, which thus may not be entirely representative. You should carefully study why there are missing labels in your evaluation data.
  warnings.warn(f"There are {n_missing} (out of {len(y_true)}) evaluation datapoints for which the label is missing. "
Evaluation: root_mean_squared_error on test data: 7.699521681198704
Evaluation: accuracy on test data: 0.69
Evaluation: accuracy on test data: 0.215
/usr/local/lib/python3.6/dist-packages/autogluon/utils/tabular/ml/learner/abstract_learner.py:360: UserWarning: There are 3 (out of 197) evaluation datapoints for which the label is missing. AutoGluon removed these points from the evaluation, which thus may not be entirely representative. You should carefully study why


Best cleaning method:
Cleaning score: Cleaner: {'outlier_detection': AutoGluonOutlierDetection, 'imputation': AutoGluonImputation}: {'roc_auc_acore': 0.809724170172978, 'classification_report': {'bad': {'precision': 0.6666666666666666, 'recall': 0.5161290322580645, 'f1-score': 0.5818181818181819, 'support': 62}, 'good': {'precision': 0.8026315789473685, 'recall': 0.8840579710144928, 'f1-score': 0.8413793103448277, 'support': 138}, 'accuracy': 0.77, 'macro avg': {'precision': 0.7346491228070176, 'recall': 0.7000935016362786, 'f1-score': 0.7115987460815048, 'support': 200}, 'weighted avg': {'precision': 0.7604824561403509, 'recall': 0.77, 'f1-score': 0.7609153605015675, 'support': 200}}} 

Cleaning improved the overall score 



CPU times: user 14min 11s, sys: 49.1 s, total: 15min
Wall time: 12min 6s


In [12]:
ind_results

[{'cleaners': [{'Imputation method': MeanModeImputation,
    'Outlier detection method': NoOutlierDetection,
    'PPP score with cleaning': {'classification_report': {'1': {'f1-score': 0.8333333333333333,
       'precision': 0.7692307692307693,
       'recall': 0.9090909090909091,
       'support': 11},
      '2': {'f1-score': 0.9259259259259259,
       'precision': 0.9615384615384616,
       'recall': 0.8928571428571429,
       'support': 28},
      'accuracy': 0.8974358974358975,
      'macro avg': {'f1-score': 0.8796296296296295,
       'precision': 0.8653846153846154,
       'recall': 0.900974025974026,
       'support': 39},
      'weighted avg': {'f1-score': 0.8998100664767332,
       'precision': 0.9072978303747535,
       'recall': 0.8974358974358975,
       'support': 39}},
     'roc_auc_acore': 0.9837662337662337}},
   {'Imputation method': MeanModeImputation,
    'Outlier detection method': PyODKNNOutlierDetection,
    'PPP score with cleaning': {'classification_report': {'1

In [15]:
cleaners = [
    (NoOutlierDetection, MeanModeImputation),
    (PyODKNNOutlierDetection, MeanModeImputation),
    (PyODKNNOutlierDetection, AutoGluonImputation),
    (PyODIsolationForestOutlierDetection, MeanModeImputation),
    (PyODIsolationForestOutlierDetection, AutoGluonImputation),
    (AutoGluonOutlierDetection, AutoGluonImputation)
]

In [16]:
%%time
for _ in range(1):
  print("\n\n..................................ITERATION..................................\n")
  ind_results = []
  
  for dataset in datasets:
    for learner, param_grid in models.items():
      for corruption in corruptions:
        for fraction in fractions:
          ind_results.append(run_experiment(dataset, learner, param_grid, corruption, fraction, cleaners, 100))



..................................ITERATION..................................



Data pickle file already exists and is up to date.


Dataset: parkinsons
Found 0 categorical and 22 numeric features 

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 174 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    3.8s finished



Generating corrupted training data on 39 rows... 

	perturbation: MissingValues: {'column': 'V2', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MNAR'}
	perturbation: MissingValues: {'column': 'V16', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MAR'}
	perturbation: MissingValues: {'column': 'V1', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MNAR'}
	perturbation: MissingValues: {'column': 'V15', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MCAR'}
	perturbation: MissingValues: {'column': 'V9', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MAR'}
	perturbation: MissingValues: {'column': 'V6', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MNAR'}
	perturbation: MissingValues: {'column': 'V17', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MCAR'}
	perturbation: MissingValues: {'column': 'V11', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MAR'}
	perturbation: MissingValues: {'column': 'V8', 'fraction': 0.15, 'na_value': nan, 'missingness': 'MCAR'}
	p

...

Cleaner: {'outlier_detection': PyODIsolationForestOutlierDetection, 'imputation': AutoGluonImputation}: {'roc_auc_acore': 0.7616877045348293, 'classification_report': {'bad': {'precision': 0.5652173913043478, 'recall': 0.41935483870967744, 'f1-score': 0.4814814814814815, 'support': 62}, 'good': {'precision': 0.7662337662337663, 'recall': 0.855072463768116, 'f1-score': 0.8082191780821918, 'support': 138}, 'accuracy': 0.72, 'macro avg': {'precision': 0.665725578769057, 'recall': 0.6372136512388967, 'f1-score': 0.6448503297818367, 'support': 200}, 'weighted avg': {'precision': 0.7039186900056466, 'recall': 0.72, 'f1-score': 0.7069304921359716, 'support': 200}}}
Column credit_history: Num NaNs: Before: 0, Now: 31
Column personal_status: Num NaNs: Before: 0, Now: 2
Column other_parties: Num NaNs: Before: 0, Now: 15
Column property_magnitude: Num NaNs: Before: 0, Now: 1
Column other_payment_plans: Num NaNs: Before: 0, Now: 38
Column housing: Num NaNs: Before: 0, Now: 39
Column own_telephone:

Evaluation: accuracy on test data: 0.455
Evaluation: root_mean_squared_error on test data: 8.26470822869023
Evaluation: accuracy on test data: 0.69
Evaluation: accuracy on test data: 0.24
/usr/local/lib/python3.6/dist-packages/autogluon/utils/tabular/ml/learner/abstract_learner.py:360: UserWarning: There are 7 (out of 193) evaluation datapoints for which the label is missing. AutoGluon removed these points from the evaluation, which thus may not be entirely representative. You should carefully study why there are missing labels in your evaluation data.
  warnings.warn(f"There are {n_missing} (out of {len(y_true)}) evaluation datapoints for which the label is missing. "
Evaluation: root_mean_squared_error on test data: 2374.5349678574485
Evaluation: accuracy on test data: 0.625
Evaluation: accuracy on test data: 0.505
Evaluation: root_mean_squared_error on test data: 0.9565251865323527
Evaluation: accuracy on test data: 0.5
Evaluation: accuracy on test data: 0.925
/usr/local/lib/python3

Cleaner: {'outlier_detection': AutoGluonOutlierDetection, 'imputation': AutoGluonImputation}: {'roc_auc_acore': 0.8142823749415615, 'classification_report': {'bad': {'precision': 0.673469387755102, 'recall': 0.532258064516129, 'f1-score': 0.5945945945945945, 'support': 62}, 'good': {'precision': 0.8079470198675497, 'recall': 0.8840579710144928, 'f1-score': 0.8442906574394464, 'support': 138}, 'accuracy': 0.775, 'macro avg': {'precision': 0.7407082038113258, 'recall': 0.7081580177653108, 'f1-score': 0.7194426260170205, 'support': 200}, 'weighted avg': {'precision': 0.7662589539126909, 'recall': 0.775, 'f1-score': 0.7668848779575425, 'support': 200}}}
Column credit_history: Num NaNs: Before: 0, Now: 31
Column personal_status: Num NaNs: Before: 0, Now: 2
Column other_parties: Num NaNs: Before: 0, Now: 15
Column property_magnitude: Num NaNs: Before: 0, Now: 1
Column other_payment_plans: Num NaNs: Before: 0, Now: 38
Column housing: Num NaNs: Before: 0, Now: 39
Column own_telephone: Num NaNs

Evaluation: accuracy on test data: 0.455
Evaluation: root_mean_squared_error on test data: 8.26470822869023
Evaluation: accuracy on test data: 0.69
Evaluation: accuracy on test data: 0.24
/usr/local/lib/python3.6/dist-packages/autogluon/utils/tabular/ml/learner/abstract_learner.py:360: UserWarning: There are 7 (out of 193) evaluation datapoints for which the label is missing. AutoGluon removed these points from the evaluation, which thus may not be entirely representative. You should carefully study why there are missing labels in your evaluation data.
  warnings.warn(f"There are {n_missing} (out of {len(y_true)}) evaluation datapoints for which the label is missing. "
Evaluation: root_mean_squared_error on test data: 2374.5349678574485
Evaluation: accuracy on test data: 0.625
Evaluation: accuracy on test data: 0.505
Evaluation: root_mean_squared_error on test data: 0.9565251865323527
Evaluation: accuracy on test data: 0.5
Evaluation: accuracy on test data: 0.925
/usr/local/lib/python3


Best cleaning method:
Cleaning score: Cleaner: {'outlier_detection': AutoGluonOutlierDetection, 'imputation': AutoGluonImputation}: {'roc_auc_acore': 0.8142823749415615, 'classification_report': {'bad': {'precision': 0.673469387755102, 'recall': 0.532258064516129, 'f1-score': 0.5945945945945945, 'support': 62}, 'good': {'precision': 0.8079470198675497, 'recall': 0.8840579710144928, 'f1-score': 0.8442906574394464, 'support': 138}, 'accuracy': 0.775, 'macro avg': {'precision': 0.7407082038113258, 'recall': 0.7081580177653108, 'f1-score': 0.7194426260170205, 'support': 200}, 'weighted avg': {'precision': 0.7662589539126909, 'recall': 0.775, 'f1-score': 0.7668848779575425, 'support': 200}}} 

Cleaning improved the overall score 



CPU times: user 46min 8s, sys: 2min 26s, total: 48min 35s
Wall time: 37min 4s


In [17]:
ind_results

[{'cleaners': [{'Imputation method': MeanModeImputation,
    'Outlier detection method': NoOutlierDetection,
    'PPP score with cleaning': {'classification_report': {'1': {'f1-score': 0.846153846153846,
       'precision': 0.7333333333333333,
       'recall': 1.0,
       'support': 11},
      '2': {'f1-score': 0.923076923076923,
       'precision': 1.0,
       'recall': 0.8571428571428571,
       'support': 28},
      'accuracy': 0.8974358974358975,
      'macro avg': {'f1-score': 0.8846153846153846,
       'precision': 0.8666666666666667,
       'recall': 0.9285714285714286,
       'support': 39},
      'weighted avg': {'f1-score': 0.9013806706114397,
       'precision': 0.9247863247863247,
       'recall': 0.8974358974358975,
       'support': 39}},
     'roc_auc_acore': 0.9967532467532467}},
   {'Imputation method': MeanModeImputation,
    'Outlier detection method': PyODKNNOutlierDetection,
    'PPP score with cleaning': {'classification_report': {'1': {'f1-score': 0.7272727272727

In [21]:
len(ind_results)

3